## Evapotranspiration Interpolation, Accuracy Assessment and PostGIS 

### Data Preparation
- Load ET data for July 2022 and July 2023.
- Prepare ground-truth data.

### Interpolation
- Perform IDW, Ordinary Kriging, and Universal Kriging interpolation.

### Accuracy Assessment
- Compare interpolated values with ground-truth data.
- Calculate RMSE, MAE, and R² for each method.

### Sending best model to Postgis
  -We will also add other models to postgis just to save the datasets



### import requires packages

In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math
from math import sqrt
import random
import psycopg2

In [3]:
arcpy.env.scratchWorkspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

with arcpy.EnvManager(scratchWorkspace=arcpy.env.scratchWorkspace):
    ETCorn = arcpy.sa.ExtractByMask(
        in_raster="ET_Jul2023.aet.tif",
        in_mask_data="clipped.TIF",
        extraction_area="INSIDE",
        analysis_extent='-97.3578140352203 43.3388462071335 -88.9958333 49.5041666666667 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
    )
    ETCorn.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExtractETCorn")


### Raster to Point

In [5]:
arcpy.conversion.RasterToPoint(
    in_raster="ETCorn",
    out_point_features=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\RetToP",
    raster_field="Value"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\RetToP'>

### ET interpolated data

In [7]:
# IDW
ET_idw = "ET2023_Idw"

# ORDINARY_KRIGING
ET_ord_kri = "ET2023_ord_krigging"

# UNIVERSAL_KRIGING
ET_uni_kri = "ET2023_uni_krigging"

### Interpolated raster to points

In [8]:
import arcpy

# Set the workspace where the output feature classes will be saved
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

# IDW
arcpy.conversion.RasterToPoint(in_raster=ET_idw, out_point_features="ET2023_IDWpoints", raster_field="Value")

# ORDINARY_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ET_ord_kri, out_point_features="ET2023_ord_Kriggingpoints", raster_field="Value")

# UNIVERSAL_KRIGING
arcpy.conversion.RasterToPoint(in_raster=ET_uni_kri, out_point_features="ET2023_uni_Kriggingpoints", raster_field="Value")


<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\ET2023_uni_Kriggingpoints'>

####  Checking Exploratort Interpolation

In [9]:
arcpy.ga.ExploratoryInterpolation(
    in_features="RetToP",
    value_field="pointid",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb\ExploratoryInterpolation",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\ExploratoryInterpolation'>

In [1]:
import arcpy
import random
from math import sqrt

# Set your workspace (where your data is stored)
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"

# Define paths to your datasets
interpolated_layer_IDW = "ET2023_IDWpoints"
interpolated_layer_ord_Krigging = "ET2023_ord_Kriggingpoints"
interpolated_layer_uni_Krigging = "ET2023_uni_Kriggingpoints"
truth_layer = "RetToP"



### Accuracy assessment 

In [2]:
def process_interpolation(interpolated_layer, truth_layer, label):
    # Read the points into memory
    interpolated_points = [row for row in arcpy.da.SearchCursor(interpolated_layer, ["SHAPE@", "grid_code"])]
    truth_points = [row for row in arcpy.da.SearchCursor(truth_layer, ["SHAPE@", "grid_code"])]

    # Function to find closest point
    def find_closest(inter_point, truth_points):
        min_distance = float('inf')
        closest_truth = None
        for truth_point in truth_points:
            dist = inter_point[0].distanceTo(truth_point[0])
            if dist < min_distance:
                min_distance = dist
                closest_truth = truth_point
        return (inter_point[1], closest_truth[1], min_distance)

    # Find closest matches
    closest_matches = [find_closest(inter_point, truth_points) for inter_point in interpolated_points]

    # Sample 50% of the data randomly
    sampled_matches = random.sample(closest_matches, len(closest_matches) // 2)

    # Function to calculate RMSE, MAE, and R²
    def calculate_metrics(data):
        true_values, predicted_values = zip(*[(true, pred) for true, pred, _ in data])
        mse = sum((t - p) ** 2 for t, p in zip(true_values, predicted_values)) / len(data)
        mae = sum(abs(t - p) for t, p in zip(true_values, predicted_values)) / len(data)
        rmse = sqrt(mse)
        mean_true = sum(true_values) / len(data)
        ss_tot = sum((t - mean_true) ** 2 for t in true_values)
        ss_res = sum((t - p) ** 2 for t, p in zip(true_values, predicted_values))
        r_squared = 1 - (ss_res / ss_tot)
        return rmse, mae, r_squared

    # Calculate and print the accuracy metrics
    rmse, mae, r_squared = calculate_metrics(sampled_matches)
    print(f"{label} RMSE: {rmse}, MAE: {mae}, R²: {r_squared}")

# Define layers
truth_layer = "RetToP"
interpolated_layer_IDW = "ET2023_IDWpoints"
interpolated_layer_ord_Krigging = "ET2023_ord_Kriggingpoints"
interpolated_layer_uni_Krigging = "ET2023_uni_Kriggingpoints"

# Process each interpolation method
process_interpolation(interpolated_layer_IDW, truth_layer, "IDW")
process_interpolation(interpolated_layer_ord_Krigging, truth_layer, "Ordinary Krigging")
process_interpolation(interpolated_layer_uni_Krigging, truth_layer, "Universal Krigging")


IDW RMSE: 1.3268690259711111, MAE: 0.7289537436107436, R²: 0.9917094515909141
Ordinary Krigging RMSE: 1.5390378125614548, MAE: 0.9850686370201813, R²: 0.9887309034932417
Universal Krigging RMSE: 1.9642631031769628, MAE: 1.1442730215360575, R²: 0.9813486706935182


### Since IDW Model seems like a best fit amongst others we are using IDW for our map and sending this to postgis

In [2]:
import arcpy
import os

# Function to convert raster to point and push to database
def point_and_push(raster_name, local_gdb_path, geodatabase_path):
    # Convert raster to point data
    arcpy.conversion.RasterToPoint(
        in_raster=os.path.join(local_gdb_path, raster_name),
        out_point_features=os.path.join(local_gdb_path, raster_name + '_point'),
        raster_field="Value"
    )
    print(f"{raster_name} converted to point")

    # Push point data to remote database
    arcpy.conversion.FeatureClassToGeodatabase(
        os.path.join(local_gdb_path, raster_name + '_point'),
        geodatabase_path
    )
    print(f"{raster_name}_point pushed to database")

# Paths
local_gdb_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Final project ET\Final project ET.gdb"
geodatabase_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Interpolation raster names
interpolation_rasters = [
    "ET2023_Idw"]

# Convert each raster to point and push to the database
for raster_name in interpolation_rasters:
    point_and_push(raster_name, local_gdb_path, geodatabase_path)


ET2023_Idw converted to point
ET2023_Idw_point pushed to database


In [3]:
import arcpy

def validate_geometry(point_feature_class):
    # Repair geometry if necessary
    arcpy.management.RepairGeometry(point_feature_class, "DELETE_NULL")

# Call this function before pushing to database
validate_geometry(os.path.join(local_gdb_path, raster_name + '_point'))


In [4]:
def set_spatial_reference(point_feature_class, srid):
    # Define the spatial reference using the SRID
    spatial_ref = arcpy.SpatialReference(srid)
    arcpy.management.DefineProjection(point_feature_class, spatial_ref)

# Example usage: Assuming SRID 4326 for WGS 84
set_spatial_reference(os.path.join(local_gdb_path, raster_name + '_point'), 4326)


In [5]:
arcpy.AddMessage("Starting conversion...")
try:
    # Your existing code here
    point_and_push(raster_name, local_gdb_path, geodatabase_path)
except Exception as e:
    arcpy.AddError(str(e))
arcpy.AddMessage("Completed conversion...")


ET2023_Idw converted to point
ET2023_Idw_point pushed to database


In [6]:
arcpy.conversion.RasterToPoint(
    in_raster=os.path.join(local_gdb_path, raster_name),
    out_point_features=os.path.join(local_gdb_path, raster_name + '_point'),
    raster_field="Value"
)


<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Final project ET\\Final project ET.gdb\\ET2023_Idw_point'>